In [1]:
import sys
import os
from pathlib import Path

# Это нужно сделать ОДИН РАЗ в начале ноутбука
# Находим корень проекта (на два уровня выше текущей директории ноутбука)
# и добавляем папку src в пути для импорта
project_root = Path.cwd().parent 
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from local_paths import RAW_DATA_DIR, FINAL_DATA_DIR
import pandas as pd

df = pd.read_parquet(FINAL_DATA_DIR / 'combined.parquet')
df.head()

,symbol,date,open,high,low,close,volume,adj_close,close_mean_3d,close_std_3d,...,close_mean_30d,close_std_30d,close_mean_60d,close_std_60d,volume_mean_7d,log_return_1d,price_range_1d,company_age_days,up_streak,down_streak
0,A,1999-11-18,32.546494,35.765381,28.612303,31.473534,62546380,26.347027,31.473534,NaN,...,31.473534,NaN,31.473534,NaN,6.254638e+07,NaN,0.227273,0,0,0
1,A,1999-11-19,30.713518,30.758226,28.478184,28.880545,15234146,24.176392,30.177039,1.833520,...,30.177039,1.833520,30.177039,1.833520,3.889026e+07,-0.085979,0.078947,1,0,1
2,A,1999-11-22,29.551144,31.473534,28.657009,31.473534,6577870,26.347027,30.609204,1.497063,...,30.609204,1.497063,30.609204,1.497063,2.811947e+07,0.085979,0.089489,4,1,0
3,A,1999-11-23,30.400572,31.205294,28.612303,28.612303,5975611,23.951838,29.655460,1.580200,...,30.109979,1.578301,30.109979,1.578301,2.258350e+07,-0.095310,0.090625,5,0,1
4,A,1999-11-24,28.701717,29.998213,28.612303,29.372318,4843231,24.588066,29.819385,1.482080,...,29.962447,1.406096,29.962447,1.406096,1.903545e+07,0.026216,0.047184,6,1,0


In [ ]:
import math
# Начальный капитал (V0) во всех случаях = 1000.0

# 1. Положительный тренд: Финал ~2000 (x2)
portfolio_uptrend = [
    1000.0, 1100, 990.0, 1032.1, 1085.6, 1120.45, 1095.3, 1140.2, 
    1185.1, 1210.4, 1195.8, 1245.3, 1280.9, 1315.4, 1290.15, 1345.6, 
    1390.25, 1425.8, 1395.1, 1455.4, 1510.3, 1565.2, 1530.45, 1590.8, 
    1655.1, 1710.4, 1785.25, 1850.6, 1915.15, 1960.3, 2005.1
]

# 2. Боковой тренд (Flat): Финал ~1000 (x1)
portfolio_flat = [
    1000.0, 900.0, 1000.0, 995.6, 978.2, 1025.15, 1034.8, 967.45, 
    992.1, 1015.3, 1045.6, 1021.1, 988.45, 965.2, 1012.9, 990.15, 
    1032.4, 975.6, 954.2, 1018.15, 1045.3, 988.7, 962.15, 1025.4, 
    1010.2, 982.15, 1035.6, 995.1, 1012.45, 985.3, 1000.0
]

# 3. Отрицательный тренд: Финал ~500 (x0.5)
portfolio_downtrend = [
    1000.0, 972.1, 984.3, 945.15, 912.4, 885.6, 902.1, 865.3, 
    834.15, 856.2, 821.4, 795.8, 772.3, 798.15, 754.6, 732.1, 
    705.45, 682.3, 712.1, 695.4, 665.15, 632.4, 610.1, 585.3, 
    615.45, 584.2, 556.1, 534.8, 512.15, 525.4, 498.9
]

print(math.e)

KOEFF_LOSE = math.e - 1  # примерно 1.718281828
def calc_reward(day_before, day_now, coef_lose=math.e - 1):
    # return (day_now - day_before) / day_before
    out = 100 * (math.log(day_now / day_before) - 0.0007188)  # с учётом инфляции ~0.0007191 в день
    if day_now < day_before:
        return coef_lose * out
    return out

for portfolio in [portfolio_uptrend, portfolio_flat, portfolio_downtrend]:
    rewards = []
    for i in range(1, len(portfolio)):
        # reward = calc_reward(portfolio[i-1], portfolio[i], 1)
        reward = calc_reward(portfolio[i-1], portfolio[i])
        rewards.append(reward)
        if i < 5:
            print(f"Награда: {portfolio[i-1]:.1f} -> {portfolio[i]:.1f} = {reward:.6f}")
    avg_reward = sum(rewards) / len(rewards)
    # print(f'Average Reward: {avg_reward:.6f}')
    procent_change = (portfolio[-1] - portfolio[0]) / portfolio[0]
    pm = '+' if procent_change >= 0 else '-'
    print(f"Итоговое изменение капитала: {portfolio[0]:.1f} -> {portfolio[-1]:.1f} ({pm}{abs(procent_change):.1%})")
    print(f"Sum Reward: {sum(rewards):.6f}\n")

2.718281828459045
Награда: 1000.0 -> 1100.0 = 9.459134
Награда: 1100.0 -> 990.0 = -18.227423
Награда: 990.0 -> 1032.1 = 4.092706
Награда: 1032.1 -> 1085.6 = 4.981843
Итоговое изменение капитала: 1000.0 -> 2005.1 (+100.5%)
Sum Reward: 52.464548

Награда: 1000.0 -> 900.0 = -18.227423
Награда: 900.0 -> 1000.0 = 10.464167
Награда: 1000.0 -> 995.6 = -0.881229
Награда: 995.6 -> 978.2 = -3.153093
Итоговое изменение капитала: 1000.0 -> 1000.0 (+0.0%)
Sum Reward: -43.871193

Награда: 1000.0 -> 972.1 = -4.985670
Награда: 972.1 -> 984.3 = 1.175321
Награда: 984.3 -> 945.1 = -7.097522
Награда: 945.1 -> 912.4 = -6.183065
Итоговое изменение капитала: 1000.0 -> 498.9 (-50.1%)
Sum Reward: -137.798527



In [46]:
# Lazy Agent Problem Solving
# Решение проблемы "ленивого агента", который выбирает действия, не влияющие на состояние среды
inflation_year = 0.30  # 30% годовых
inflation_day = (1 + inflation_year)**(1/365) - 1
inflation_week  = (1 + inflation_day)**7 - 1
inflation_month = (1 + inflation_day)**30 - 1
inflation_year_check = (1 + inflation_day)**365 - 1
print(f"Lazy Agent Problem Solving:")
print(f"- Все деньги, которые не инвестированы, сжигаются инфляцией, в среднем {inflation_year:.1%} в год")
print(f"- Если робот ничего не делает, его капитал будет ежедневно уменьшаться")

labels = ["День:", "Неделя:", "Месяц:", "Год:"]
values = [inflation_day, inflation_week, inflation_month, inflation_year_check]

print(f"Точные метрики (Compound Interest):")

# Используем ширину 10 символов для меток
print(f"{labels[0]:10} {values[0]:.5%}")
print(f"{labels[1]:10} {values[1]:.5%}")
print(f"{labels[2]:10} {values[2]:.5%}")
print(f"{labels[3]:10} {values[3]:.5%} (Проверка)")

print(f"Дневной Коэффициент для вставки в формулу награды: {inflation_day:.7f}")
print(f"Предрасчет дневного коэффициента через логарифм: {math.log(1 + inflation_day):.7f}")

Lazy Agent Problem Solving:
- Все деньги, которые не инвестированы, сжигаются инфляцией, в среднем 30.0% в год
- Если робот ничего не делает, его капитал будет ежедневно уменьшаться
Точные метрики (Compound Interest):
День:      0.07191%
Неделя:    0.50443%
Месяц:     2.17984%
Год:       30.00000% (Проверка)
Дневной Коэффициент для вставки в формулу награды: 0.0007191
Предрасчет дневного коэффициента через логарифм: 0.0007188


```python
# Вместо линейного деления (inflation_year / 365)
# Используйте геометрическое среднее для корректного логарифмического сложения
inflation_day = (1 + inflation_year)**(1/365) - 1
```

**Обновленная логика награды:**
$$R_t = \ln\left(\frac{V_t}{V_{t-1}}\right) - \ln(1 + inflation\_day)$$
Эта формула автоматически делает "нулевой" награду, если рост портфеля в точности равен инфляции.